In [261]:
import pandas as pd

In [262]:
data = pd.read_csv("Chloe_Full_Health.csv")

/var/folders/8t/gvv4mvq55nv9hrww0yk38zv80000gn/T/ipykernel_43984/3979618993.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("Chloe_Full_Health.csv")


In [263]:
data.head()

,Unnamed: 0,type,unit,startDate,value,Date,Day,ActivityType
0,0,HKQuantityTypeIdentifierHeight,ft,2024-01-19 01:11:52 -0500,5.75,2024-01-19,Friday,Height
1,1,HKQuantityTypeIdentifierHeartRate,count/min,2021-10-21 01:58:02 -0500,80.0,2021-10-21,Thursday,HeartRate
2,2,HKQuantityTypeIdentifierHeartRate,count/min,2021-10-21 01:59:03 -0500,76.5744,2021-10-21,Thursday,HeartRate
3,3,HKQuantityTypeIdentifierHeartRate,count/min,2021-10-21 02:01:39 -0500,76.0,2021-10-21,Thursday,HeartRate
4,4,HKQuantityTypeIdentifierHeartRate,count/min,2021-10-21 02:04:49 -0500,74.0,2021-10-21,Thursday,HeartRate


In [264]:
data = data.loc[:, ["value", "Date", "ActivityType"]]

In [265]:
data.shape

(1545669, 3)

In [266]:
data["ActivityType"].unique()

array(['Height', 'HeartRate', 'RespiratoryRate', 'StepCount',
       'DistanceWalkingRunning', 'BasalEnergyBurned',
       'ActiveEnergyBurned', 'FlightsClimbed', 'DietaryFatTotal',
       'DietaryFatSaturated', 'DietarySodium', 'DietarySugar',
       'DietaryProtein', 'DietaryVitaminD', 'DietaryIron',
       'AppleExerciseTime', 'DietaryCaffeine', 'DistanceSwimming',
       'SwimmingStrokeCount', 'RestingHeartRate',
       'WalkingHeartRateAverage', 'DistanceDownhillSnowSports',
       'HeadphoneAudioExposure', 'WalkingDoubleSupportPercentage',
       'SixMinuteWalkTestDistance', 'AppleStandTime', 'WalkingSpeed',
       'WalkingStepLength', 'WalkingAsymmetryPercentage',
       'AppleWalkingSteadiness', 'HKCategoryTypeIdentifierSleepAnalysis',
       'HKCategoryTypeIdentifierAppleStandHour',
       'HKCategoryTypeIdentifierMindfulSession',
       'HeartRateVariabilitySDNN'], dtype=object)

In [267]:
select = ['HeartRate', 'RespiratoryRate', 'RestingHeartRate', 'WalkingAsymmetryPercentage']
data = data[data['ActivityType'].isin(select)]

In [268]:
data

,value,Date,ActivityType
1,80.0,2021-10-21,HeartRate
2,76.5744,2021-10-21,HeartRate
3,76.0,2021-10-21,HeartRate
4,74.0,2021-10-21,HeartRate
5,74.0,2021-10-21,HeartRate
...,...,...,...
1517920,0,2024-04-11,WalkingAsymmetryPercentage
1517921,0,2024-04-12,WalkingAsymmetryPercentage
1517922,0,2024-04-12,WalkingAsymmetryPercentage
1517923,0.06,2024-04-12,WalkingAsymmetryPercentage


In [269]:
data.shape

(269322, 3)

In [270]:
data

,value,Date,ActivityType
1,80.0,2021-10-21,HeartRate
2,76.5744,2021-10-21,HeartRate
3,76.0,2021-10-21,HeartRate
4,74.0,2021-10-21,HeartRate
5,74.0,2021-10-21,HeartRate
...,...,...,...
1517920,0,2024-04-11,WalkingAsymmetryPercentage
1517921,0,2024-04-12,WalkingAsymmetryPercentage
1517922,0,2024-04-12,WalkingAsymmetryPercentage
1517923,0.06,2024-04-12,WalkingAsymmetryPercentage


In [271]:
data['Date'] = pd.to_datetime(data['Date'])

In [272]:
# Ensure 'Value' column contains numeric values
data['value'] = pd.to_numeric(data['value'], errors='coerce')

In [273]:
# Check for non-numeric values in 'Value' column
non_numeric_mask = data['value'].isna()
if non_numeric_mask.any():
    print("Warning: There are non-numeric values in the 'Value' column. These rows will be dropped.")


In [274]:
data = data.dropna(subset= ['value'])

In [275]:
average_values = data.groupby(['Date', 'ActivityType'])['value'].mean().reset_index(name = "AverageValue")

In [276]:
average_values.shape

(3347, 3)

In [277]:
average_values

,Date,ActivityType,AverageValue
0,2021-10-11,WalkingAsymmetryPercentage,0.000000
1,2021-10-12,WalkingAsymmetryPercentage,0.006667
2,2021-10-13,WalkingAsymmetryPercentage,0.007647
3,2021-10-14,WalkingAsymmetryPercentage,0.000000
4,2021-10-15,WalkingAsymmetryPercentage,0.006250
...,...,...,...
3342,2024-04-11,RestingHeartRate,67.000000
3343,2024-04-11,WalkingAsymmetryPercentage,0.100000
3344,2024-04-12,HeartRate,96.119467
3345,2024-04-12,RestingHeartRate,60.000000


In [278]:
average_values["ActivityType"].unique()

array(['WalkingAsymmetryPercentage', 'RestingHeartRate', 'HeartRate',
       'RespiratoryRate'], dtype=object)

In [279]:
# Define thresholds
thresholds = {
    'HeartRate': 100,
    'RespiratoryRate': 20,
    'WalkingAsymmetryPercentage': 15
}

In [280]:
# Iterate through the data and add a message if thresholds are exceeded
for index, row in average_values.iterrows():
    if row['ActivityType'] in thresholds and row['AverageValue'] > thresholds[row['ActivityType']]:
        average_values.at[index, 'Message'] = "Consider taking a mental health break."


In [281]:
average_values.to_csv('output-mental-stress.csv')

In [282]:
# Apply threshold conditions for each activity type
thresholded_data = pd.concat([average_values[average_values['ActivityType'] == activity_type][average_values['AverageValue'] > threshold]
                              for activity_type, threshold in thresholds.items()])

print(thresholded_data)

           Date     ActivityType  AverageValue  \
11   2021-10-21        HeartRate    104.729802   
36   2021-10-28        HeartRate    109.579878   
40   2021-10-29        HeartRate    101.539930   
84   2021-11-09        HeartRate    105.681533   
92   2021-11-11        HeartRate    103.974096   
...         ...              ...           ...   
3311 2024-04-02  RespiratoryRate     21.000000   
3318 2024-04-04  RespiratoryRate     21.375000   
3322 2024-04-05  RespiratoryRate     20.500000   
3326 2024-04-06  RespiratoryRate     23.400000   
3333 2024-04-08  RespiratoryRate     25.000000   

                                     Message  
11    Consider taking a mental health break.  
36    Consider taking a mental health break.  
40    Consider taking a mental health break.  
84    Consider taking a mental health break.  
92    Consider taking a mental health break.  
...                                      ...  
3311  Consider taking a mental health break.  
3318  Consider taking a

/var/folders/8t/gvv4mvq55nv9hrww0yk38zv80000gn/T/ipykernel_43984/2847974071.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  thresholded_data = pd.concat([average_values[average_values['ActivityType'] == activity_type][average_values['AverageValue'] > threshold]


In [283]:
from collections import defaultdict
import pandas as pd
from datetime import datetime, timedelta
import calendar

def predict_stressful_week(assignments):
    stress_keywords = ["test", "deadline", "presentation", "meeting"]
    event_counts_per_day = defaultdict(int)
    current_week = datetime.now().isocalendar()[1]  # Get the current week number

    # Convert "Due Date" column to datetime
    assignments["Due Date"] = pd.to_datetime(assignments["Due Date"], errors="coerce")

    for index, assignment in assignments.iterrows():
        # Extracting the name and due date from the assignment
        name = assignment["Name"]
        due_date = assignment["Due Date"]

        # Checking if the assignment is due the same week
        if pd.notnull(due_date) and due_date.isocalendar()[1] == current_week:
            event_counts_per_day[due_date.date()] += 1

        # Checking for stress keywords in the name
        if any(keyword in name.lower() for keyword in stress_keywords):
            event_counts_per_day[due_date.date()] += 1

    # Checking if there are too many stressful events in the week
    stressful_days = [day for day, count in event_counts_per_day.items() if count >= 3]

    # If there are stressful days, add a message to the DataFrame
    if len(stressful_days) > 0:
        assignments["Message"] = "Your week looks a bit stressful. Consider making time for yourself."

    return assignments

# Example usage:
assignments_df = pd.read_csv("assignments.csv")  # Assuming the file contains "Name" and "Due Date" columns

# Predict stressful week and update DataFrame
assignments_with_message = predict_stressful_week(assignments_df)

# Save the DataFrame to a CSV file
assignments_with_message.to_csv("assignments_with_message.csv", index=False)